# Collegedata scraping (1)

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

import requests
import time
from bs4 import BeautifulSoup
import re
import TIdatabase as ti

In [2]:
genderdict = {'Male': -1, 'Female': 1}
highschooldict = {'Public': -1, 'Private': 1, 'Parochial': 1, 'Home-Schooled': 1}
minoritylist = ['african', 'hispanic', 'latin','indian', 'native',  ]
nonminoritylist = ['white', 'caucasian', 'asian', 'european', 'chinese']
sats=[1600, 1560, 1510, 1460, 1420, 1380, 1340, 1300, 1260, 1220, 1190, 1150, 1110, 1070, 1030, 990, 950, 910, 870, 
      830, 790, 740, 690, 640, 590, 530]
act2satdict=dict(zip(range(11,37),sats))
booleandict={'': -1, 'X': 1}
statusdict={'Will Attend': 1, 'Accepted': 1, 'Applied': 0, 'Deferred': -1, 'Denied': -1, 'Not Applied': 0, 'Wait-Listed': 1, 'Withdrawn': 0}

In [3]:
def getFromDict(dictionary,text):
    if dictionary.has_key(text):
        return dictionary[text]
    else:
        return 0
def isMinority(text):
    for m in minoritylist:
        if m in text:
            return 1
    for m in nonminoritylist:
        if m in text:
            return -1
    print text
    return 0

In [4]:
college_ids=['Princeton', 'Harvard', 'Yale', 'Columbia', 'Stanford', 'UChicago', 'MIT', 'Duke', 'UPenn', 'CalTech', 'JohnsHopkins', 'Dartmouth', 'Northwestern', 'Brown', 'Cornell', 'Vanderbilt', 'WashU', 'Rice', 'NotreDame', 'UCB', 'Emory', 'Georgetown', 'CarnegieMellon', 'UCLA', 'USC']
college_urls=[111, 444, 244, 399, 781, 327, 186, 1026, 67, 706, 1509, 403, 1803, 163, 787, 1562, 1720, 731, 1774, 1090, 1039, 1182, 204, 1093, 1138]
college_id_dict=dict(zip(college_ids,college_urls))
uni_list=['Princeton University', 'Harvard College', 'Yale University', 'Columbia University', 'Stanford University', 'University of Chicago', 'Massachusetts Institute of Technology', 'Duke University', 'University of Pennsylvania', 'California Institute of Technology', 'Johns Hopkins University', 'Dartmouth College', 'Northwestern University', 'Brown University', 'Cornell University', 'Vanderbilt University', 'Washington University in St. Louis', 'Rice University', 'University of Notre Dame', 'University of California, Berkeley', 'Emory Univesrity', 'Georgetown University', 'Carnegie Mellon University', 'University of California, Los Angeles', 'University of Southern California']
uni_name_dict = dict(zip(uni_list, college_ids))
uni_state=['NJ', 'MA', 'CT', 'NY', 'CA', 'IL', 'MA', 'NC', 'PA', 'CA', 'MD', 'NH', 'IL', 'RI', 'NY', 'TN', 'MO', 'TX', 'IN', 'CA', 'GA', 'DC', 'PA', 'CA', 'CA']
states_dict={'Alabama': 'AL',
 'Alaska': 'AK',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'National': 'NA',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Northern Mariana Islands': 'MP',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virgin Islands': 'VI',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY',
  'Other': 'Other'}
uni_state_dict = dict(zip(uni_list,uni_state))
baseurl='http://www.collegedata.com/cs/admissions/'
tracker_url='admissions_tracker_result.jhtml?schoolId='
student_url='admissions_profile_view.jhtml?profileName='

In [5]:
## Source: https://impythonist.wordpress.com/2015/01/06/ultimate-guide-for-scraping-javascript-rendered-web-pages/
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 

class Render(QWebPage):  
    def __init__(self, url):  
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()  
  
    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit()

In [6]:
schoolurl=baseurl+tracker_url+str(444)+'&classYear='
r=Render(schoolurl+str(2014))
result = r.frame.toHtml()
#print str(result.toAscii())

In [7]:
soup = BeautifulSoup(str(result.toAscii()),'html.parser')
#r = requests.get(schoolurl+str(2009))
#time.sleep(3)
#soup = BeautifulSoup(r.text,'html.parser')
#print soup.prettify()
#rows = soup.find(id="profilewrap") #.find(id='tableBodyWrap').find_all("tr")[1:]
#print rows

In [8]:
profiles= soup.find_all("a",href=re.compile(r"enterProfileByName"))
profileList=set()
for p in profiles:
    profileList.add(p.get("href").split("'")[1])
print len(profileList)

89


In [9]:
def getScores(scores,fun):
    scores = [s.get_text().strip() for s in scores]
    while '' in scores:
        scores.remove('')
    if len(scores)>0:
        scores=[int(s) for s in scores]
        return fun(scores), len(scores)
    else:
        return None, 0
def getAdmissionTestScore(doc):
    satCRM,dummy = getScores(doc[0:2],sum)
    satW,dummy = getScores([doc[2]],np.max)
    act,dummy = getScores([doc[4]],np.max)
    if act == None:
        return (satCRM+satW)
    elif satCRM==None:
        if satW==None:
            print "Warning: no SAT writing score"
        return act2satdict[act]+satW
    else:
        return (max(act2satdict[act],satCRM)+satW)

In [10]:
columns_student = ['classrank', 'admissionstest','AP','averageAP','SATsubject', 'GPA', 'program','schooltype',
            'intendedgradyear', 'canAfford', 'female', 'MinorityGender','MinorityRace','international',
           'firstinfamily','sports','artist', 'workexp']
columns_uni = ['collegeID','earlyAppl','visited','alumni', 'outofstate', 'acceptStatus','acceptProb']

In [11]:
def getColumnValues(soup):
    values=dict(zip(columns_student,[None for i in range(len(columns_student))]))
    applications=[]
    # We start with the general information box at the top which includes class year, gender and ethnicity
    doc= soup.find("div",{"class": "general"})
    values['intendedgradyear'] = int(re.findall(r'\d{4}',doc.find("h1").get_text().split('Class of')[-1])[0]) # CLASS YEAR
    doc = doc.find_all("span")
    values['female']= getFromDict(genderdict,doc[0].get_text().strip()) # GENDER
    values['MinorityGender']= 1 if values['female']==0 else -1
    values['MinorityRace'] = isMinority(doc[1].get_text().strip().lower()) # MINORITY RACE
    values['program'] = doc[2].get_text().strip() # PROGRAM
    # Now we look at the academics box which includes GPA and high school info
    doc = soup.find("div", {"class": "academicswrap"}).find_all("span")
    values['schooltype']=getFromDict(highschooldict,doc[0].get_text().strip()) # SCHOOL TYPE
    state=states_dict[doc[1].get_text().strip()] # STATE OF STUDENT
    values['international'] = 1 if state=='Other' else -1
    values['GPA'] = float(doc[3].get_text()) # This is the unweighted GPA, to get weighted, change to doc[4]
    # Next, we go to the test score box which includes SAT, ACT and AP info
    values['admissionstest'] = getAdmissionTestScore(soup.find("div", {"class": "testscorewrap"}).find_all("td")) #Admissions test
    values['SATsubject'] = len(soup.find("caption",text="SAT Subject Test Scores").next_sibling.next_sibling.find_all("tr")) # SAT SUBJECT
    ap_num = len(soup.find("caption",text="AP Examinations").next_sibling.next_sibling.find_all("tr"))
    values['AP']=ap_num #AP
    if ap_num>0:
        doc = soup.find("caption",text="AP Examinations").next_sibling.next_sibling.find_all("td")
        values['averageAP'],values['AP']= getScores(doc,np.mean) # AVERAGE AP
    doc = soup.find_all("div", {"class": "word"})
    doc = [d.get_text().strip() for d in doc]  #TODO add to new column in database
    #print doc[0]+doc[1]+doc[2]
    # Next: the colleges applied to and the admission results for the admissions table
    doc = soup.find("table", {"class": "collchoice"})
    collegelist = doc.find_all("tr")[2:]
    for c in collegelist:
        uni = c.find("th").get_text().strip()
        if uni in uni_list:
            unirow = dict(zip(columns_uni,[None for i in range(6)]))
            unirow['collegeID']=uni_name_dict[uni]
            doc=c.find_all("td", {"class": "center"})
            unirow['earlyAppl']=booleandict[doc[0].get_text().strip()] # Early Admission
            unirow['alumni']=booleandict[doc[1].get_text().strip()] # Alumni/Legacy
            if values['sports']==None or values['sports']==0: 
                values['sports']=booleandict[doc[2].get_text().strip()] # Athlete
            doc = doc[2].next_sibling.find_next("span")
            unirow['acceptStatus']=statusdict[doc.get_text().strip()] # Admission status
            if values['canAfford']==None or values['canAfford']==0:
                values['canAfford']=-1 if 'Yes' in doc.find_next("span").get_text().strip() else 0
            unirow['outofstate']= -1 if state==uni_state_dict[uni] else 1
            applications.append(unirow)
    return values, applications

In [12]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applForm' in locals()): del applForm
students=ti.Student()
colleges = ti.College()
applForm = ti.ApplForm()
for p in profileList:
    #print p
    profile_url=baseurl+student_url+p
    soup=BeautifulSoup(requests.get(profile_url).text,'html.parser')
    newrow, applications = getColumnValues(soup)
    studentID=students.insert(newrow)
    for app in applications:
        app['studentID']=studentID[0]
    applForm.insert(applications)
students.df


jamaican

hebrew
polish


albanian


black
multiracial

multiracial
middle eastern
n/a


mexican/native american
black
caucasion





,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp
0,HC2Y3PHO5R,None,2280,2,4.5,3,3.75,"Engineering, Economics",-1,2014,0,-1,-1,-1,-1,None,-1,None,None
0,ED9N3NYSXV,None,2120,0,None,3,3.75,,-1,2014,0,1,-1,-1,1,None,-1,None,None
0,PC5X3M24TM,None,2340,1,5,3,4.00,,1,2014,0,-1,-1,-1,-1,None,-1,None,None
0,CIT3FC858W,None,2250,7,4.285714,2,3.80,,-1,2014,0,1,-1,0,-1,None,-1,None,None
0,SQ5XXBCFG7,None,2280,0,None,3,3.90,Mathematics,1,2014,0,-1,-1,-1,1,None,-1,None,None
0,EOQ6WJ2OBP,None,2290,3,5,2,4.00,finance/economics/biology/computer science,1,2014,0,-1,-1,-1,-1,None,-1,None,None
0,PEA80AXPPN,None,1970,3,2.333333,3,3.40,"Economics/Foreign Languages(Spanish, Chinese)",-1,2014,0,-1,-1,0,-1,None,-1,None,None
0,T64O47RU8K,None,2340,7,5,4,4.00,Biology,1,2014,0,-1,-1,-1,-1,None,-1,None,None
0,1OTYE6SZKV,None,2220,7,4.428571,4,3.76,Politics,-1,2014,0,-1,-1,1,-1,None,-1,None,None
0,ZMNNLSCJ8I,None,2210,2,5,3,3.80,Political Science,-1,2014,0,-1,-1,-1,-1,None,-1,None,None


In [15]:
applForm.df

,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,IRET4VOKYJ,Princeton,-1,None,-1,1,1,NaN
1,IRET4VOKYJ,Yale,-1,None,-1,1,-1,NaN
2,IRET4VOKYJ,Columbia,-1,None,-1,1,1,NaN
3,IRET4VOKYJ,Harvard,-1,None,1,1,1,NaN
4,IRET4VOKYJ,Cornell,-1,None,-1,1,-1,NaN
5,IRET4VOKYJ,UCB,-1,None,-1,1,1,NaN
6,IRET4VOKYJ,UCLA,-1,None,-1,1,1,NaN
7,IRET4VOKYJ,Georgetown,-1,None,-1,1,1,NaN
0,RK0AMGQ0WQ,UPenn,-1,None,-1,1,-1,NaN
1,RK0AMGQ0WQ,Princeton,-1,None,-1,1,-1,NaN


In [24]:
#p=list(profileList)[12]
#p='3q9x4'
p='meggymack92'
#p='6FNsQ'
profile_url=baseurl+student_url+p
print profile_url
soup=BeautifulSoup(requests.get(profile_url).text,'html.parser')
#print soup.prettify()
#print columns
print getColumnValues(soup)

http://www.collegedata.com/cs/admissions/admissions_profile_view.jhtml?profileName=meggymack92
ELD Tutor- Secretary (1 year, 3 hrs/wk)School Valedictorian, Varsity Letter, Gold Medal at SkillsUSA State Conference in Technical Computer Applications (both years), 6th Place at National SkillsUSA Technical Computer Applications Competition, Principal's Honor Roll, AP Scholar, FFA Greenhand Degree,
({'intendedgradyear': 2015, 'classrank': None, 'canAfford': None, 'schooltype': -1, 'artist': None, 'MinorityGender': 0, 'averageAP': 3.1428571428571428, 'admissionstest': 531, 'GPA': 4.0, 'sports': -1, 'AP': 7, 'program': u'Psychology', 'female': 1, 'workexp': None, 'international': -1, 'MinorityRace': -1, 'SATsubject': 4, 'firstinfamily': None}, [{'acceptStatus': -1, 'alumni': -1, 'earlyAppl': -1, 'outofstate': 1, 'visited': None, 'collegeID': 'Princeton'}, {'acceptStatus': -1, 'alumni': -1, 'earlyAppl': -1, 'outofstate': 1, 'visited': None, 'collegeID': 'Harvard'}, {'acceptStatus': 1, 'alumni'